In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [3]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## statsmodels（先頭30フレームの平均）

In [3]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995516  0.584104  0.274504  0.461335  0.312321  0.185005   
question2     0.995715  0.642856  0.404782  0.516417  0.306051  0.173076   
question3     0.996076  0.649879  0.343308  0.467093  0.331270  0.158914   
question4     0.996124  0.614563  0.311653  0.455995  0.334674  0.176307   
question5     0.995153  0.623812  0.276110  0.575167  0.328001  0.308602   
question6     0.994046  0.619062  0.364727  0.511042  0.441243  0.228932   
question7     0.994499  0.646352  0.345298  0.471837  0.404303  0.205212   
question8     0.996331  0.577665  0.332649  0.453592  0.310819  0.294546   
question9     0.997603  0.716374  0.393647  0.480373  0.367472  0.221597   
question10    0.996228  0.737441  0.409118  0.466749  0.376113  0.197430   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.986745  0.677718  0.635501  0.335615  0.405039  0.141602   
question2     0.987059  0.464171  0.303665  0.657852  0.415585  0.168717   
question3     0.973344  0.593614  0.569758  0.360023  0.537402  0.216448   
question4     0.968963  0.577749  0.573284  0.269166  0.533981  0.232647   
question5     0.966330  0.652138  0.538727  0.462535  0.508015  0.181007   
question6     0.955797  0.635025  0.557141  0.409525  0.673926  0.105880   
question7     0.976715  0.521238  0.404524  0.518077  0.537137  0.083637   
question8     0.939318  0.588318  0.519676  0.266799  0.580388  0.268763   
question9     0.966117  0.647312  0.579556  0.361143  0.508877  0.220936   
question10    0.950850  0.630409  0.510022  0.317216  0.593382  0.189391   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

In [30]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3487
Time:                        23:40:49   Log-Likelihood:                -123.16
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.195e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.1498      2.804      3.263      0.001       3.654      14.645
AU01         -10.4605      2.943     -3.554      0.000     -16.229      -4.692
AU02           8.8264

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3615
Time:                        23:40:49   Log-Likelihood:                -121.04
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 8.027e-21
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.9889      2.867      3.484      0.000       4.370      15.608
AU01         -10.2891      2.961     -3.474      0.001     -16.093      -4.485
AU02           9.3156      2.432      3.831      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3544
Time:                        23:40:49   Log-Likelihood:                -122.07
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 3.114e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.1333      2.846      3.209      0.001       3.555      14.712
AU01         -10.2704      2.980     -3.447      0.001     -16.111      -4.430
AU02           8.8156      2.409      3.660      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3429
Time:                        23:40:50   Log-Likelihood:                -120.79
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 1.079e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.1089      2.805      3.248      0.001       3.611      14.606
AU01          -9.4094      2.965     -3.174      0.002     -15.220      -3.599
AU02           7.9005      2.415      3.272      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3506
Time:                        23:40:51   Log-Likelihood:                -119.37
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 3.076e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.5210      2.911      2.584      0.010       1.816      13.226
AU01          -8.4863      3.045     -2.787      0.005     -14.454      -2.518
AU02           8.1977      2.417      3.392      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3447
Time:                        23:40:51   Log-Likelihood:                -123.91
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.606e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.4392      2.791      3.382      0.001       3.970      14.909
AU01         -10.0513      2.962     -3.393      0.001     -15.857      -4.246
AU02           8.6784      2.399      3.617      0.0

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.13028095480044982, 0.2337769708742009, 0.13000506399559955, 0.08516094696381747, 0.1981067820643861, 0.21265048163455558, 0.20016699270339064, 0.2040233701323959, 0.1782458138160236, 0.25765373059506197]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.040297185366412386, 0.014137468242823885, 0.003880131880410603, 0.034337131231365184, 0.018157446884275653, 0.04911698310052098, 0.012742358223572631, 0.12444118421156071, 0.021328047878342065, 0.01410366504332761]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 71.43%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 1, 1, 1, 1, 0]
クラス1の確率: [0.3356725475616961, 0.8830487473021097, 0.5042771079563435, 0.9720339667661593, 0.8335893785820848, 0.7422243023677898, 0.49202853838867133]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 20.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5704770155225548, 0.5249286197907147, 0.23204710332368955, 0.3923800249375609, 0.3033887275189211, 0.32927743328520187, 0.32254287998861336, 0.3363492948577315, 0.32292313111928306, 0.26674703000452404]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 11.11%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3419213130769407, 0.5292503302095395, 0.10015234044005045, 0.2414335995038904,

## statsmodels（先頭10フレームの平均）

In [5]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.994253  0.548917  0.309042  0.369101  0.340054  0.205853   
question2     0.995213  0.673667  0.509078  0.513324  0.264832  0.195403   
question3     0.996800  0.666825  0.336399  0.492604  0.332760  0.160899   
question4     0.996812  0.619893  0.317726  0.485552  0.308422  0.169937   
question5     0.995148  0.601538  0.277234  0.589133  0.330110  0.325815   
question6     0.995141  0.619397  0.362123  0.533383  0.458388  0.232431   
question7     0.994641  0.678903  0.357197  0.486914  0.381950  0.224525   
question8     0.997165  0.556887  0.305754  0.475889  0.310583  0.270285   
question9     0.998061  0.690526  0.395891  0.488248  0.373891  0.224995   
question10    0.996726  0.717515  0.432269  0.437019  0.448252  0.194634   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.997739  0.444869  0.279833  0.596534  0.409172  0.233146   
question2     0.997044  0.360181  0.229466  0.708506  0.401036  0.168001   
question3     0.994291  0.460411  0.286429  0.521735  0.304848  0.258847   
question4     0.989487  0.568114  0.452583  0.399706  0.425778  0.265401   
question5     0.992415  0.601455  0.417037  0.456763  0.402993  0.349553   
question6     0.994112  0.533990  0.413505  0.563756  0.369665  0.306439   
question7     0.996506  0.698964  0.518986  0.538167  0.356811  0.174849   
question8     0.989735  0.613677  0.449690  0.446160  0.384153  0.335018   
question9     0.991478  0.572113  0.446910  0.485027  0.372616  0.403829   
question10    0.990900  0.527835  0.322207  0.685897  0.333907  0.387484   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [32]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2754
Time:                        23:41:00   Log-Likelihood:                -137.00
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.493e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.9167      2.421      2.444      0.015       1.172      10.662
AU01          -6.0039      2.275     -2.639      0.008     -10.464      -1.544
AU02           5.2865

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2609
Time:                        23:41:00   Log-Likelihood:                -137.38
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 3.193e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.9832      2.412      2.481      0.013       1.256      10.710
AU01          -5.4454      2.261     -2.408      0.016      -9.877      -1.013
AU02           5.2161      1.851      2.819      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2735
Time:                        23:41:01   Log-Likelihood:                -137.73
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.834e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.6692      2.439      2.735      0.006       1.890      11.449
AU01          -5.8328      2.294     -2.543      0.011     -10.328      -1.337
AU02           4.7939      1.848      2.594      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2769
Time:                        23:41:01   Log-Likelihood:                -136.74
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.186e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1117      2.422      2.523      0.012       1.364      10.859
AU01          -5.0256      2.270     -2.214      0.027      -9.476      -0.576
AU02           4.9747      1.857      2.679      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  286
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2829
Time:                        23:41:01   Log-Likelihood:                -136.65
converged:                       True   LL-Null:                       -190.56
Covariance Type:            nonrobust   LLR p-value:                 3.123e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.9379      2.408      2.466      0.014       1.219      10.657
AU01          -6.1075      2.283     -2.675      0.007     -10.582      -1.633
AU02           5.7346      1.881      3.048      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2769
Time:                        23:41:02   Log-Likelihood:                -132.92
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 4.162e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4940      2.491      1.804      0.071      -0.389       9.377
AU01          -3.6774      2.331     -1.578      0.115      -8.246       0.891
AU02           4.7986      1.861      2.579      0.0

ファイル名: filtered_ID69.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 28/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2847
Time:                        23:41:02   Log-Likelihood:                -132.23
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 9.420e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.7918      2.450      2.772      0.006       1.99

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3539926807678074, 0.42670230733998626, 0.21561024762867398, 0.09736290462206386, 0.3501881511036978, 0.32790700652931976, 0.30806277567987106, 0.3093622891453991, 0.39787760457814203, 0.47613631296748765]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.1907092808139585, 0.060248093235972154, 0.03614378701885088, 0.16740106982830585, 0.12886484586828134, 0.14963736278093318, 0.08884401616869311, 0.11157098513698169, 0.15526690807461063, 0.08654592716129267]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 85.71%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 1, 1, 1, 1, 1]
クラス1の確率: [0.2970349087091595, 0.8915662710559079, 0.7612653004256764, 0.9070729507547776, 0.5864617244137503, 0.7380588491369957, 0.5194762582180003]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 10.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.44462410977450006, 0.41585295191130495, 0.2196823551867395, 0.36850886907110947, 0.2519972382692522, 0.3018012419088287, 0.262711861989331, 0.5469507712660024, 0.3318577591606739, 0.3349840385019879]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3097956524121784, 0.37666919982021063, 0.2593765266742956, 0.27380301220266323, 0.24

## statsmodels（先頭30フレームの標準偏差）

In [7]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001466  0.082011  0.085878  0.133645  0.058596  0.050013   
question2     0.001270  0.081372  0.124629  0.117510  0.064030  0.045970   
question3     0.001708  0.075652  0.092508  0.122632  0.077482  0.030568   
question4     0.001270  0.083811  0.072418  0.098150  0.072119  0.032883   
question5     0.001197  0.071634  0.093603  0.136910  0.053622  0.084010   
question6     0.001881  0.063219  0.086607  0.123548  0.075216  0.048818   
question7     0.002001  0.067252  0.083081  0.136454  0.096351  0.042810   
question8     0.001401  0.068174  0.114510  0.134832  0.034390  0.065531   
question9     0.001172  0.070727  0.085490  0.118371  0.076434  0.032543   
question10    0.001385  0.052577  0.074468  0.165595  0.090610  0.033810   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question2     0.000762  0.055500  0.093741  0.049720  0.059136  0.028451   
question3     0.000660  0.026932  0.061047  0.090022  0.099477  0.014659   
question4     0.000137  0.027041  0.064017  0.086725  0.050755  0.010834   
question5     0.000105  0.047570  0.064736  0.105519  0.084890  0.009841   
question6     0.000189  0.028350  0.054353  0.087920  0.082841  0.012693   
question7     0.000656  0.033582  0.056839  0.132640  0.083888  0.017988   
question8     0.000213  0.034925  0.050481  0.098690  0.062816  0.007926   
question9     0.000485  0.097362  0.059464  0.113796  0.057746  0.026320   
question10    0.000365  0.074656  0.078169  0.152889  0.051044  0.010193   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                               ...                       
quest

In [34]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2886
Time:                        23:41:11   Log-Likelihood:                -134.52
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.752e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8601      0.759     -5.085      0.000      -5.348      -2.372
AU01          -2.0448      6.142     -0.333      0.739     -14.084       9.994
AU02         -12.1479

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2775
Time:                        23:41:11   Log-Likelihood:                -136.62
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.069e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5710      0.757     -4.717      0.000      -5.055      -2.087
AU01          -5.1352      6.313     -0.813      0.416     -17.508       7.238
AU02          -9.5626      6.469     -1.478      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2672
Time:                        23:41:11   Log-Likelihood:                -138.92
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 5.060e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6564      0.759     -4.817      0.000      -5.144      -2.169
AU01          -1.3680      6.092     -0.225      0.822     -13.307      10.571
AU02         -10.6983      6.385     -1.676      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2592
Time:                        23:41:12   Log-Likelihood:                -140.07
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.051e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8633      0.747     -5.171      0.000      -5.328      -2.399
AU01          -1.7743      6.115     -0.290      0.772     -13.760      10.211
AU02         -11.7028      6.480     -1.806      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  286
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3072
Time:                        23:41:12   Log-Likelihood:                -132.02
converged:                       True   LL-Null:                       -190.56
Covariance Type:            nonrobust   LLR p-value:                 5.575e-17
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.8758      0.842     -5.789      0.000      -6.527      -3.225
AU01          -6.4685      6.518     -0.992      0.321     -19.243       6.306
AU02          -8.6337      6.646     -1.299      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2776
Time:                        23:41:12   Log-Likelihood:                -132.77
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 3.676e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5871      0.764     -4.695      0.000      -5.084      -2.090
AU01          -7.0333      6.568     -1.071      0.284     -19.907       5.841
AU02         -11.6033      6.730     -1.724      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2468
Time:                        23:41:12   Log-Likelihood:                -139.22
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 3.602e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9824      0.762     -5.223      0.000      -5.477      -2.488
AU01          -2.8707      6.223     -0.461      0.645     -15.068       9.327
AU02         -10.0296      6.395     -1.568      0.1

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.44101037658374365, 0.3274809354354973, 0.46751530776236117, 0.3364328003860398, 0.29541874146852565, 0.2585505428719184, 0.3680511526531451, 0.4033173011696401, 0.2615920155303323, 0.3942316067041245]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5359795543884516, 0.23826101854194562, 0.20651009938172193, 0.2229080708866572, 0.3654195366633454, 0.4584670925239921, 0.41618851674003843, 0.30240098027039264, 0.3715941078912488, 0.47384925496030444]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 14.29%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.35864246484630014, 0.3481055662490947, 0.40783121014080664, 0.4188461233947435, 0.3458173088163296, 0.3561733499688799, 0.5070577788007986]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35545501516244915, 0.35663633857299687, 0.3554251729158551, 0.35419128294733565, 0.35620142523293225, 0.3561691033859704, 0.3547802656861296, 0.3549220256599306, 0.3560024666229967, 0.35440877786345953]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.2183469044852629, 0.20306714370180126, 0.24660725207008183, 0.22484457134529684

## statsmodels（先頭10フレームの標準偏差）

In [26]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001121  0.093524  0.112813  0.113783  0.070513  0.058268   
question2     0.001382  0.089263  0.136364  0.145956  0.027679  0.049899   
question3     0.000877  0.048792  0.071889  0.116778  0.077599  0.029961   
question4     0.000845  0.095508  0.052701  0.104434  0.062406  0.040550   
question5     0.001609  0.054508  0.082077  0.167779  0.059787  0.087103   
question6     0.001885  0.068061  0.087083  0.142162  0.062328  0.051092   
question7     0.001011  0.066099  0.094583  0.117106  0.093062  0.055414   
question8     0.000721  0.064097  0.123955  0.131484  0.025264  0.039280   
question9     0.000666  0.086447  0.063176  0.087117  0.092931  0.036451   
question10    0.001114  0.055978  0.052930  0.178878  0.065129  0.033687   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.007996  0.087572  0.132737  0.146465  0.062704  0.119484   
question2     0.005135  0.132575  0.124925  0.211021  0.076990  0.170071   
question3     0.011776  0.045625  0.109528  0.242119  0.106644  0.079748   
question4     0.019453  0.053513  0.053514  0.079920  0.129792  0.091010   
question5     0.018167  0.080723  0.109186  0.145466  0.118800  0.093223   
question6     0.010932  0.104754  0.067611  0.130933  0.087272  0.035644   
question7     0.006455  0.182245  0.176745  0.202367  0.092513  0.009990   
question8     0.020566  0.100134  0.114468  0.089797  0.135974  0.160679   
question9     0.022744  0.084196  0.144495  0.172806  0.134686  0.119861   
question10    0.016807  0.106081  0.166942  0.123167  0.143464  0.084387   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

In [27]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.2047
Time:                        02:41:41   Log-Likelihood:                -150.39
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.104e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1133      0.573     -5.431      0.000      -4.237      -1.990
AU01          -4.6202      4.908     -0.941      0.347     -14.240       4.999
AU02          -3.7777

ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1742
Time:                        02:41:42   Log-Likelihood:                -153.50
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 1.689e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2454      0.573     -5.668      0.000      -4.36

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1893
Time:                        02:41:42   Log-Likelihood:                -153.70
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.070e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0849      0.572     -5.392      0.000      -4.206      -1.964
AU01          -4.9311      4.925     -1.001      0.317     -14.585       4.723
AU02          -3.0630      4.680     -0.655      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.2292
Time:                        02:41:43   Log-Likelihood:                -141.68
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 6.655e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.7191      0.610     -6.098      0.000      -4.914      -2.524
AU01          -1.7434      4.898     -0.356      0.722     -11.344       7.857
AU02          -3.3403      4.761     -0.702      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.2504
Time:                        02:41:43   Log-Likelihood:                -141.75
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.522e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.0951      0.634     -6.457      0.000      -5.338      -2.852
AU01          -7.0579      5.232     -1.349      0.177     -17.313       3.198
AU02          -2.8581      4.968     -0.575      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1934
Time:                        02:41:44   Log-Likelihood:                -149.10
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 1.191e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0880      0.575     -5.367      0.000      -4.216      -1.960
AU01          -4.7965      4.918     -0.975      0.329     -14.436       4.843
AU02          -0.9326      4.707     -0.198      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.2013
Time:                        02:41:45   Log-Likelihood:                -146.81
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 4.386e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1597      0.590     -5.355      0.000      -4.316      -2.003
AU01          -6.7561      5.193     -1.301      0.193     -16.934       3.422
AU02          -4.5173      4.838     -0.934      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1950
Time:                        02:41:45   Log-Likelihood:                -152.62
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 4.494e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2433      0.572     -5.673      0.000      -4.364      -2.123
AU01          -3.5927      4.871     -0.738      0.461     -13.139       5.954
AU02          -3.8824      4.813     -0.807      0.4

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 1, 0, 0, 1, 0, 0, 0, 1]
クラス1の確率: [0.5387256652737628, 0.13982867538278063, 0.6120449787347533, 0.2913222066863886, 0.46148566747397846, 0.5285637444716954, 0.42520050678120136, 0.46142982060310794, 0.2911396567068561, 0.6413913610383585]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.7206344796931973, 0.25059928861882785, 0.15327671532033554, 0.2508341629677848, 0.25606868359727525, 0.17115990763126762, 0.5542722124549843, 0.22382723663383902, 0.2797735902914251, 0.4496328500849985]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 22.22%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 1, 0]
クラス1の確率: [0.39623490906932335, 0.19492339711389103, 0.2717295731271121, 0.38255924124045393, 0.4745854910404855, 0.7727631775756083, 0.4320524074996945, 0.7213794578384336, 0.3919029601286765]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35932503719238174, 0.36017175139538005, 0.3640114823657001, 0.3606368293556536, 0.35875704559431965, 0.35889651767664266, 0.3660453678868249]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3557501096040352, 0.35610066959204695, 0.35428665966932116, 0.3541426663563292, 0.356207487070

## statsmodels（先頭30フレームの差分）

In [4]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.001724 -0.251561 -0.248012  0.076567 -0.139892  0.030934   
2     question1    0.000788  0.066084  0.033166 -0.046987  0.005213 -0.070424   
3     question1   -0.000408  0.121311  0.016596  0.155112  0.035684  0.133026   
4     question1    0.000278 -0.113827 -0.060162  0.081835 -0.004229 -0.135589   
5     question1    0.001025  0.059989  0.139242 -0.150123 -0.069870  0.067338   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.000181  0.001104  0.085981  0.045987 -0.157633  0.066186   
296  question10   -0.001860  0.067276 -0.032094 -0.116374  0.043166 -0.022014   
297  question10    0.002063 -0.006732 -0.072016  0.150383 -0.033546 -0.031211   
298  question10   -0.003481 -0.029945  0.094034 -0.154169  0.187788  0.032302   
299  question10   -0.001027 -0.039412 -0.024116  0.091600 -0.018093 -0.039674 

          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.003006  0.209979 -0.017758 -0.276734 -0.000377 -0.063774   
2     question1   -0.003928  0.144772  0.259707 -0.153668  0.175676 -0.068777   
3     question1    0.005734 -0.097903 -0.098664  0.427041 -0.165956 -0.008098   
4     question1    0.001867  0.086274  0.162361 -0.324375 -0.015583  0.119927   
5     question1    0.000082 -0.131505 -0.173601  0.206946 -0.015158  0.036716   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.012779  0.049457  0.399001 -0.103701 -0.028394 -0.042054   
296  question10   -0.015853 -0.073158 -0.209708 -0.041191  0.103534  0.038424   
297  question10    0.015451  0.007041 -0.135395  0.232145 -0.024501  0.027644   
298  question10   -0.007127 -0.037868  0.091544 -0.171391  0.045963 -0.025800   
299  question10   -0.002391  0.009290  0.066618 -0.077592 -0.092290  0.166105   

     AU07      AU09      AU

In [36]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               6.284e-05
Time:                        23:41:21   Log-Likelihood:                -5458.0
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4425      0.023    -19.499      0.000      -0.487      -0.398
AU01          -0.0542      0.251     -0.216      0.829      -0.546       0.438
AU02           0.0181

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               6.603e-05
Time:                        23:41:21   Log-Likelihood:                -5337.7
converged:                       True   LL-Null:                       -5338.0
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5847      0.023    -25.352      0.000      -0.630      -0.539
AU01          -0.0498      0.255     -0.195      0.845      -0.549       0.450
AU02           0.0289      0.205      0.141      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8198
Model:                          Logit   Df Residuals:                     8180
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               5.131e-05
Time:                        23:41:22   Log-Likelihood:                -5478.9
converged:                       True   LL-Null:                       -5479.2
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4509      0.023    -19.903      0.000      -0.495      -0.407
AU01          -0.0484      0.249     -0.195      0.846      -0.536       0.439
AU02           0.0211      0.202      0.104      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               5.581e-05
Time:                        23:41:22   Log-Likelihood:                -5472.4
converged:                       True   LL-Null:                       -5472.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4483      0.023    -19.779      0.000      -0.493      -0.404
AU01          -0.0604      0.250     -0.241      0.809      -0.551       0.430
AU02           0.0262      0.201      0.130      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               6.124e-05
Time:                        23:41:22   Log-Likelihood:                -5472.4
converged:                       True   LL-Null:                       -5472.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4483      0.023    -19.778      0.000      -0.493      -0.404
AU01          -0.0426      0.250     -0.170      0.865      -0.533       0.448
AU02           0.0231      0.202      0.115      0.9

...

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8167
Model:                          Logit   Df Residuals:                     8149
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               7.311e-05
Time:                        23:41:23   Log-Likelihood:                -5463.4
converged:                       True   LL-Null:                       -5463.8
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4447      0.023    -19.604      0.000      -0.489      -0.400
AU01          -0.0453      0.256     -0.177      0.859      -0.546       0.456
AU02           0.0325      0.205      0.158      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               5.625e-05
Time:                        23:41:23   Log-Likelihood:                -5458.1
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4425      0.023    -19.500      0.000      -0.487      -0.398
AU01          -0.0379      0.255     -0.149      0.882      -0.538       0.462
AU02           0.0208      0.206      0.101      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               5.908e-05
Time:                        23:41:23   Log-Likelihood:                -5307.8
converged:                       True   LL-Null:                       -5308.1
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5946      0.023    -25.700      0.000      -0.640      -0.549
AU01          -0.0458      0.263     -0.174      0.862      -0.561       0.469
AU02           0.0280      0.207      0.135      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8138
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               6.573e-05
Time:                        23:41:23   Log-Likelihood:                -5458.0
converged:                       True   LL-Null:                       -5458.4
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4425      0.023    -19.498      0.000      -0.487      -0.398
AU01          -0.0451      0.250     -0.181      0.857      -0.534       0.444
AU02           0.0263      0.203      0.129      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 8185
Model:                          Logit   Df Residuals:                     8167
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:               6.418e-05
Time:                        23:41:24   Log-Likelihood:                -5472.4
converged:                       True   LL-Null:                       -5472.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4483      0.023    -19.778      0.000      -0.493      -0.404
AU01          -0.0407      0.253     -0.161      0.872      -0.536       0.455
AU02           0.0258      0.205      0.126      0.9

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID34.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID36.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID69.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID73.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0

## statsmodels（先頭30フレームの差分の標準偏差）

In [9]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001357  0.109357  0.108198  0.137026  0.068213  0.064572   
question2     0.001140  0.100844  0.111314  0.155546  0.080165  0.067363   
question3     0.001540  0.101435  0.111410  0.189246  0.102810  0.040152   
question4     0.001450  0.103023  0.112387  0.138747  0.081161  0.039937   
question5     0.001605  0.113691  0.124395  0.193588  0.075903  0.102474   
question6     0.001716  0.081146  0.126423  0.151933  0.080299  0.062055   
question7     0.002743  0.079252  0.112768  0.150238  0.107974  0.056872   
question8     0.001929  0.095910  0.145074  0.185013  0.050982  0.097036   
question9     0.001081  0.103380  0.134461  0.168435  0.103486  0.049545   
question10    0.001231  0.073412  0.098630  0.219724  0.112046  0.047315   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000744  0.157039  0.119311  0.281567  0.060357  0.081511   
question2     0.003587  0.221230  0.202328  0.332368  0.059899  0.135979   
question3     0.012542  0.224759  0.245685  0.306997  0.061761  0.046894   
question4     0.000583  0.211562  0.198678  0.249539  0.040874  0.015306   
question5     0.000688  0.101010  0.197050  0.247647  0.095170  0.077281   
question6     0.000931  0.258910  0.284972  0.323969  0.071011  0.066707   
question7     0.000944  0.259236  0.257171  0.381766  0.070389  0.052065   
question8     0.000663  0.265806  0.247573  0.399171  0.062448  0.050194   
question9     0.000536  0.224894  0.255449  0.383642  0.070612  0.075426   
question10    0.000849  0.246835  0.238140  0.283037  0.064103  0.012654   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

In [38]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2998
Time:                        23:41:32   Log-Likelihood:                -132.41
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.806e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2199      0.763     -5.532      0.000      -5.715      -2.725
AU01           3.2556      4.833      0.674      0.501      -6.218      12.729
AU02          -9.9727

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2647
Time:                        23:41:33   Log-Likelihood:                -139.04
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.553e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2208      0.757     -5.578      0.000      -5.704      -2.738
AU01           1.7401      4.792      0.363      0.717      -7.653      11.133
AU02          -8.6625      4.792     -1.808      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2657
Time:                        23:41:33   Log-Likelihood:                -138.86
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 7.297e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1514      0.765     -5.424      0.000      -5.651      -2.651
AU01           1.7691      4.781      0.370      0.711      -7.602      11.140
AU02          -8.5634      4.804     -1.783      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3132
Time:                        23:41:33   Log-Likelihood:                -126.94
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 9.682e-17
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.7029      0.789     -5.959      0.000      -6.250      -3.156
AU01           0.5201      4.985      0.104      0.917      -9.250      10.290
AU02          -7.3928      4.934     -1.498      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2603
Time:                        23:41:34   Log-Likelihood:                -135.96
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 5.532e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2231      0.781     -5.404      0.000      -5.755      -2.692
AU01           3.7399      4.819      0.776      0.438      -5.705      13.185
AU02          -9.6417      4.911     -1.963      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2842
Time:                        23:41:34   Log-Likelihood:                -131.58
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 1.316e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9698      0.771     -5.150      0.000      -5.481      -2.459
AU01          -3.9570      5.380     -0.736      0.462     -14.501       6.587
AU02          -9.5731      5.179     -1.849      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2547
Time:                        23:41:34   Log-Likelihood:                -137.77
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 1.067e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.3097      0.760     -5.668      0.000      -5.800      -2.820
AU01           2.3658      4.881      0.485      0.628      -7.201      11.933
AU02          -8.3690      4.797     -1.745      0.0

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 50.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
クラス1の確率: [0.6105304706952727, 0.5362749464269339, 0.7854835436341594, 0.49188825959084825, 0.3417572189365734, 0.22121686454545814, 0.4329350038617874, 0.5783295078815658, 0.24449651406153985, 0.5557875291407025]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.47196004357406135, 0.25259256815664327, 0.21214097571469395, 0.17043296624693902, 0.25133705172929877, 0.49577220581254966, 0.3185318727170387, 0.3955256354520231, 0.423108827288974, 0.34470668935497195]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 14.29%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.36741727202107416, 0.3283737106184715, 0.43996001319899675, 0.41066458796026184, 0.32158988717752923, 0.30320715959335914, 0.5554405664789804]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.357475634804709, 0.3604410893092714, 0.35810552472739754, 0.35296110534007774, 0.35962651015963637, 0.3609697680294156, 0.35676894563521383, 0.35657124872197, 0.3591975952969404, 0.35518116100274544]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 11.11%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.13150696289922248, 0.16154097620176874, 0.14252820598256274, 0.0567282659286

## statsmodels（先頭30フレームの最大値）

In [11]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998051  0.806033  0.480008  0.717420  0.461953  0.308946   
question2     0.998089  0.767921  0.681266  0.755417  0.462867  0.325665   
question3     0.998311  0.795240  0.590224  0.725626  0.549736  0.252454   
question4     0.998152  0.756844  0.466964  0.659100  0.487703  0.237110   
question5     0.997102  0.767689  0.505109  0.819110  0.420315  0.500384   
question6     0.998499  0.725317  0.590334  0.698781  0.572304  0.347129   
question7     0.997608  0.784583  0.525491  0.776084  0.581872  0.351664   
question8     0.998259  0.683937  0.604178  0.745114  0.400974  0.502183   
question9     0.998850  0.825670  0.604052  0.809606  0.493420  0.267617   
question10    0.998399  0.836448  0.547312  0.800809  0.504607  0.276122   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995992  0.831859  0.802384  0.664591  0.488839  0.375927   
question2     0.996227  0.739314  0.752008  0.861240  0.708188  0.528054   
question3     0.987261  0.818295  0.764862  0.829465  0.746883  0.805030   
question4     0.986941  0.723356  0.748016  0.625498  0.708273  0.474847   
question5     0.990534  0.831801  0.794213  0.685720  0.733943  0.425925   
question6     0.981771  0.783178  0.779802  0.742087  0.765348  0.354484   
question7     0.988004  0.842255  0.808278  0.748271  0.699656  0.139837   
question8     0.977810  0.734891  0.769971  0.589003  0.738916  0.620431   
question9     0.997326  0.799541  0.797023  0.729258  0.752196  0.473217   
question10    0.990630  0.797317  0.748189  0.567522  0.763235  0.659013   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [40]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2765
Time:                        23:41:42   Log-Likelihood:                -136.81
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.264e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8807      2.970     -0.633      0.527      -7.703       3.941
AU01          -2.4016      3.038     -0.791      0.429      -8.356       3.553
AU02           5.4756

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3310
Time:                        23:41:43   Log-Likelihood:                -126.49
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.573e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0458      3.102     -0.659      0.510      -8.127       4.035
AU01           0.8287      3.149      0.263      0.792      -5.343       7.000
AU02           7.2705      1.940      3.748      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2764
Time:                        23:41:43   Log-Likelihood:                -136.83
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.281e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8672      2.986     -0.960      0.337      -8.720       2.986
AU01          -0.3161      2.996     -0.106      0.916      -6.188       5.556
AU02           5.3355      1.769      3.015      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.3358
Time:                        23:41:43   Log-Likelihood:                -122.77
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 2.494e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.6609      3.320     -1.705      0.088     -12.168       0.846
AU01          -0.9390      3.136     -0.299      0.765      -7.086       5.208
AU02           4.6358      1.901      2.439      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2760
Time:                        23:41:44   Log-Likelihood:                -133.09
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 4.796e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4434      3.023     -0.477      0.633      -7.369       4.483
AU01          -1.9803      3.036     -0.652      0.514      -7.931       3.970
AU02           5.8187      1.804      3.225      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  286
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2623
Time:                        23:41:44   Log-Likelihood:                -137.88
converged:                       True   LL-Null:                       -186.90
Covariance Type:            nonrobust   LLR p-value:                 2.052e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3528      2.941     -0.800      0.424      -8.117       3.411
AU01          -1.2939      2.962     -0.437      0.662      -7.100       4.512
AU02           5.5058      1.793      3.071      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2787
Time:                        23:41:44   Log-Likelihood:                -136.39
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 8.786e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5800      3.026     -1.183      0.237      -9.510       2.350
AU01          -0.6358      2.988     -0.213      0.831      -6.491       5.220
AU02           5.9196      1.794      3.299      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2799
Time:                        23:41:44   Log-Likelihood:                -136.16
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 7.196e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0482      2.954     -0.693      0.488      -7.837       3.741
AU01          -0.1211      3.018     -0.040      0.968      -6.036       5.794
AU02           5.0832      1.777      2.860      0.0

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
クラス1の確率: [0.3901353233998334, 0.5470135631699116, 0.3401662583739921, 0.21384203341866237, 0.5394898306820528, 0.33072878230477576, 0.41478741308649775, 0.6068078749505049, 0.33967177798188486, 0.443786141845268]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.501589272585878, 0.1983553470504234, 0.18918992455662703, 0.21506186437248845, 0.35930378143682296, 0.2108187008803615, 0.22095814733280947, 0.47478731600938173, 0.29840231579265974, 0.23870711157636362]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0,

最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.437574983919036, 0.41924528552937584, 0.4621652041561771, 0.493090989202416, 0.4584511991892712, 0.4751008173829326, 0.4726694464581309]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40095668803262413, 0.45270913857596556, 0.3890655458693953, 0.38668385764962215, 0.4392579703636763, 0.43177451047960513, 0.3306310155958703, 0.3372571716062034, 0.3821282538439556, 0.32966841845826866]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24814877574652344, 0.16328119362570828, 0.15752244320977252, 0.14665536573853727, 

## statsmodels（先頭10フレームの最大値）

In [13]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.996113  0.675807  0.480008  0.545058  0.461953  0.308946   
question2     0.996619  0.767921  0.681266  0.755417  0.338528  0.325665   
question3     0.998311  0.744807  0.426928  0.673520  0.474646  0.200453   
question4     0.997968  0.749176  0.402691  0.659100  0.433518  0.232112   
question5     0.997069  0.664431  0.374271  0.819110  0.390757  0.500384   
question6     0.998499  0.712010  0.469338  0.695301  0.572304  0.347129   
question7     0.996357  0.784583  0.514673  0.598215  0.571546  0.351664   
question8     0.998259  0.677315  0.544166  0.745114  0.359447  0.314839   
question9     0.998850  0.786054  0.473068  0.628629  0.493420  0.267617   
question10    0.998399  0.790493  0.485790  0.800809  0.504607  0.251513   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.997319  0.869002  0.743316  0.606473  0.323643  0.227748   
question2     0.999806  0.822252  0.792973  0.906311  0.378102  0.108450   
question3     0.999604  0.838317  0.818678  0.660992  0.353966  0.119443   
question4     0.999671  0.856561  0.795868  0.663552  0.400649  0.125859   
question5     0.999436  0.823355  0.722341  0.623536  0.319620  0.114718   
question6     0.999282  0.839738  0.607996  0.712526  0.321833  0.139829   
question7     0.999503  0.772689  0.554541  0.671143  0.325684  0.072068   
question8     0.999611  0.816255  0.647205  0.745535  0.331183  0.082752   
question9     0.996253  0.887339  0.838096  0.399012  0.320816  0.102836   
question10    0.992222  0.889932  0.817714  0.676701  0.353984  0.071879   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995178  0.695640  0.468742  0.763997  0.415821  0.322477   
question2     0.998520  0.741546  0.786375  0.618556  0.450743  0.240929   
question3     0.999048  0.693991  0.805505  0.471664  0.458733  0.263494   
question4          NaN       NaN       NaN       NaN       NaN       NaN   
question5     0.999538  0.777253  0.286859  0.875951  0.383800  0.208503   
question6     0.999373  0.748734  0.665383  0.720392  0.467512  0.190700   
question7     0.999642  0.618924  0.358839  0.676483  0.424730  0.201275   
question8     0.999239  0.678733  0.699998  0.597113  0.581831  0.342862   
question9     0.999016  0.737939  0.802950  0.510493  0.440405  0.188224   
question10    0.999020  0.750304  0.821293  0.353198  0.407962  0.300125   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [42]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2584
Time:                        23:41:53   Log-Likelihood:                -140.23
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 2.354e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6892      2.245     -0.307      0.759      -5.089       3.710
AU01          -3.0736      2.253     -1.364      0.172      -7.489       1.342
AU02           3.5801

ファイル名: filtered_ID37.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2464
Time:                        23:41:53   Log-Likelihood:                -142.88
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.457e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3073      2.247     -0.137      0.891      -4.71

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2513
Time:                        23:41:54   Log-Likelihood:                -137.62
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 2.258e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1160      2.310      0.050      0.960      -4.411       4.643
AU01          -2.3880      2.279     -1.048      0.295      -6.854       2.078
AU02           3.2569      1.436      2.267      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2661
Time:                        23:41:54   Log-Likelihood:                -138.77
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 6.770e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1166      2.275     -0.051      0.959      -4.575       4.341
AU01          -3.3425      2.305     -1.450      0.147      -7.861       1.176
AU02           3.6963      1.449      2.551      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2620
Time:                        23:41:54   Log-Likelihood:                -136.42
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 3.404e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6353      2.290     -0.277      0.781      -5.124       3.853
AU01          -2.9690      2.263     -1.312      0.190      -7.405       1.467
AU02           3.3732      1.438      2.346      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2535
Time:                        23:41:55   Log-Likelihood:                -141.16
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 5.162e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4124      2.300      0.179      0.858      -4.095       4.919
AU01          -2.9731      2.232     -1.332      0.183      -7.347       1.401
AU02           3.8622      1.442      2.679      0.0

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
クラス1の確率: [0.4562386492160722, 0.4788926332045276, 0.4396839400095397, 0.13454823114490844, 0.7244391293608168, 0.36669106831549275, 0.3715025964394344, 0.6477837835262751, 0.26837687816811356, 0.48734902506599775]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4850627177354039, 0.19346041750478035, 0.1377549184400031, 0.16760981900096952, 0.275755683500048, 0.15376925178895356, 0.31524277106032134, 0.3827575646306669, 0.2415404018091751, 0.21984171446966938]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 22.22%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 1, 0]
クラス1の確率: [0.005710698812250794, 0.012035417027694747, 0.17088223173285313, 0.13996237695928546, 0.49575355823706385, 0.6955915280967635, 0.10288916552634955, 0.6860115344567163, 0.0894794020120839]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42129319375499197, 0.4638696335044559, 0.48615506527974844, 0.45344607065142767, 0.43508316692957977, 0.4549335157215877, 0.4627616493100111]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 20.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.38338578575055915, 0.49300773797685904, 0.39361146977298384, 0.5755074451378995, 0.355639335

## statsmodels（先頭30フレームの最大値と最小値の差分）

In [15]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.005760  0.390331  0.345453  0.500715  0.214697  0.221865   
question2     0.005446  0.281593  0.459288  0.494674  0.225900  0.216869   
question3     0.007495  0.276103  0.377743  0.456334  0.303219  0.155282   
question4     0.006056  0.333635  0.316722  0.382445  0.232637  0.119928   
question5     0.005688  0.258043  0.350116  0.502948  0.186523  0.326944   
question6     0.008136  0.235677  0.383564  0.483693  0.333962  0.187738   
question7     0.009249  0.275935  0.323897  0.608640  0.328619  0.236012   
question8     0.005283  0.257964  0.457597  0.555496  0.163178  0.310530   
question9     0.004680  0.308041  0.403254  0.582350  0.254546  0.109945   
question10    0.005946  0.201322  0.287763  0.667440  0.250951  0.135159   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1          NaN       NaN       NaN       NaN       NaN       NaN   
question2     0.003975  0.265499  0.434314  0.249903  0.227991  0.109232   
question3     0.002769  0.102010  0.255330  0.418583  0.256125  0.049455   
question4     0.000514  0.113175  0.275156  0.322057  0.147354  0.047624   
question5     0.000425  0.183456  0.250766  0.388175  0.238482  0.045895   
question6     0.001101  0.126074  0.224783  0.350593  0.218365  0.050997   
question7     0.002894  0.160936  0.233419  0.541948  0.248697  0.066896   
question8     0.000819  0.158908  0.202654  0.351187  0.230469  0.032811   
question9     0.002459  0.382481  0.217895  0.437068  0.203196  0.119688   
question10    0.002018  0.255208  0.317488  0.536478  0.223595  0.040225   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [44]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2768
Time:                        23:42:07   Log-Likelihood:                -136.76
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.207e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.7303      0.823     -4.530      0.000      -5.344      -2.116
AU01          -1.7225      1.729     -0.996      0.319      -5.111       1.666
AU02          -2.4818

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  285
Model:                          Logit   Df Residuals:                      267
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2475
Time:                        23:42:08   Log-Likelihood:                -139.88
converged:                       True   LL-Null:                       -185.88
Covariance Type:            nonrobust   LLR p-value:                 2.648e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9156      0.825     -4.747      0.000      -5.532      -2.299
AU01          -1.7796      1.726     -1.031      0.303      -5.163       1.604
AU02          -2.0710      1.861     -1.113      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2604
Time:                        23:42:08   Log-Likelihood:                -140.21
converged:                       True   LL-Null:                       -189.58
Covariance Type:            nonrobust   LLR p-value:                 1.526e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8117      0.829     -4.596      0.000      -5.437      -2.186
AU01          -1.4308      1.717     -0.833      0.405      -4.797       1.935
AU02          -2.5639      1.875     -1.367      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2985
Time:                        23:42:09   Log-Likelihood:                -128.95
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 1.362e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.0931      0.862     -4.749      0.000      -5.782      -2.404
AU01          -0.0722      1.781     -0.041      0.968      -3.563       3.419
AU02          -1.3861      1.982     -0.699      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  286
Model:                          Logit   Df Residuals:                      268
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2971
Time:                        23:42:09   Log-Likelihood:                -133.95
converged:                       True   LL-Null:                       -190.56
Covariance Type:            nonrobust   LLR p-value:                 2.983e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.7675      0.887     -5.372      0.000      -6.507      -3.028
AU01          -2.5092      1.783     -1.407      0.159      -6.004       0.985
AU02          -1.3848      1.908     -0.726      0.4

ファイル名: filtered_ID56.csv
予測率: 28.57%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 1, 0, 0, 0]

===== テストデータ: ファイル 24/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2638
Time:                        23:42:09   Log-Likelihood:                -135.33
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 3.238e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5714      0.825     -4.328      0.000      -5.189      -1.954
AU01

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2794
Time:                        23:42:10   Log-Likelihood:                -136.26
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 7.886e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5469      0.854     -4.155      0.000      -5.220      -1.874
AU01           0.5562      1.894      0.294      0.769      -3.157       4.269
AU02          -2.9122      1.876     -1.553      0.1

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42120038737409893, 0.39842390380189774, 0.3862188346715952, 0.27905892279088307, 0.28677392853592015, 0.27147885689639545, 0.3267586333707598, 0.4058615504513033, 0.2495043855983832, 0.3713474972287931]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
クラス1の確率: [0.6690959822959919, 0.3599790102755576, 0.19853130093503218, 0.20118769809169007, 0.49627653373339464, 0.4293412566235653, 0.3144131797532593, 0.3118737648074511, 0.5482691037817313, 0.6815586927812517]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 66.67%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 1, 0, 0, 1, 1, 1, 1]
クラス1の確率: [0.4342180443546943, 0.5117079134716166, 0.9216248587936255, 0.20865730148253855, 0.21866381623829603, 0.9394188136137853, 0.6142909139060957, 0.8155892798340246, 0.6218138771192214]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 14.29%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.3593323820263475, 0.33124443027272205, 0.4322287991798437, 0.41393915266275993, 0.3539801806222768, 0.33924898728350356, 0.506015247466374]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3753779944505872, 0.3853428745711984, 0.37724145751163196, 0.36573964851657476, 0.384680349022006

## xgboost（先頭10フレームの最大値と最小値の差分）

In [17]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.003822  0.260105  0.302495  0.328353  0.208069  0.194241   
question2     0.003976  0.267006  0.360994  0.494674  0.101561  0.178335   
question3     0.002594  0.142312  0.174857  0.404229  0.207329  0.089944   
question4     0.002409  0.257331  0.168647  0.344704  0.178452  0.114930   
question5     0.005654  0.154785  0.214451  0.458863  0.156966  0.278399   
question6     0.005794  0.210950  0.262568  0.480213  0.194533  0.183369   
question7     0.003267  0.174082  0.288023  0.403695  0.301149  0.205485   
question8     0.002394  0.223722  0.365346  0.419062  0.071020  0.123186   
question9     0.002268  0.268426  0.201086  0.270355  0.254546  0.109945   
question10    0.003167  0.155367  0.132764  0.588269  0.223793  0.110550   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001074  0.114054  0.257839  0.334237  0.163070  0.058622   
question2     0.000602  0.261794  0.473031  0.326558  0.229371  0.038325   
question3     0.001331  0.302574  0.469401  0.266180  0.186112  0.083496   
question4     0.000695  0.122128  0.245469  0.247078  0.152864  0.064509   
question5     0.000495  0.246611  0.461765  0.231013  0.198171  0.064357   
question6     0.001521  0.038748  0.075222  0.309542  0.139359  0.196776   
question7     0.001030  0.196114  0.216387  0.386164  0.104848  0.026544   
question8     0.000989  0.127582  0.283845  0.341830  0.097172  0.250121   
question9     0.001042  0.393650  0.502502  0.471779  0.160830  0.030501   
question10    0.002607  0.105215  0.195803  0.469666  0.203753  0.065128   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [46]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1948
Time:                        23:42:19   Log-Likelihood:                -152.26
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 4.994e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0768      0.590     -5.214      0.000      -4.233      -1.920
AU01          -1.2722      1.587     -0.802      0.423      -4.382       1.837
AU02          -0.9733

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1753
Time:                        23:42:20   Log-Likelihood:                -155.94
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 9.228e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1347      0.590     -5.311      0.000      -4.292      -1.978
AU01          -1.8217      1.599     -1.139      0.255      -4.957       1.313
AU02          -0.4872      1.524     -0.320      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2178
Time:                        23:42:20   Log-Likelihood:                -143.78
converged:                       True   LL-Null:                       -183.81
Covariance Type:            nonrobust   LLR p-value:                 3.740e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5779      0.619     -5.778      0.000      -4.791      -2.364
AU01          -0.0697      1.638     -0.043      0.966      -3.281       3.141
AU02          -0.7043      1.571     -0.448      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.2470
Time:                        23:42:21   Log-Likelihood:                -142.39
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 1.467e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2198      0.666     -6.334      0.000      -5.525      -2.914
AU01          -1.8121      1.732     -1.046      0.295      -5.207       1.582
AU02          -0.8591      1.659     -0.518      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  284
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1957
Time:                        23:42:21   Log-Likelihood:                -148.67
converged:                       True   LL-Null:                       -184.85
Covariance Type:            nonrobust   LLR p-value:                 8.426e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.7675      0.624     -6.038      0.000      -4.990      -2.544
AU01          -0.8805      1.599     -0.551      0.582      -4.014       2.253
AU02          -0.7083      1.541     -0.460      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  283
Model:                          Logit   Df Residuals:                      265
Method:                           MLE   Df Model:                           17
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                  0.1798
Time:                        23:42:21   Log-Likelihood:                -155.08
converged:                       True   LL-Null:                       -189.09
Covariance Type:            nonrobust   LLR p-value:                 4.734e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1654      0.593     -5.337      0.000      -4.328      -2.003
AU01          -1.1693      1.584     -0.738      0.460      -4.273       1.935
AU02          -0.2131      1.539     -0.138      0.8

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 1, 0, 0, 0, 0, 1, 0, 1]
クラス1の確率: [0.5507238632204887, 0.21180001956148548, 0.6707359304529826, 0.28589955133286643, 0.462381241561693, 0.4563689983901071, 0.41557309598656716, 0.5302799129078543, 0.2814880155901046, 0.6199852210494237]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.6298637942632155, 0.28668579842318226, 0.2232176210073863, 0.267548991321942, 0.3000748504939312, 0.22237407682459984, 0.41030543538451264, 0.28270073673535856, 0.2603845260051981, 0.3238517882397004]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36263791980371574, 0.3528668912517684, 0.375512889236154, 0.3756349496493974, 0.3661635038977935, 0.39758798088725295, 0.36252034897016383, 0.3882648680226587, 0.36103395895698365]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3577446393249342, 0.36041271341423853, 0.3832592443268973, 0.36296907184196364, 0.35109810597225666, 0.35166652393413417, 0.3910659107492846]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36868750529989985, 0.3730958634879005, 0.35831177015001375, 0.35826198019104777, 0.37422480283